### Streaming Twitter Dashboard

In this notebook, we extract the real time streaming twitter data and plot keyword specific sentiments across US states. This data can be analysed by a Social Media manager to create campaigns based on which states to target for which product. 

In [1]:
import os
os.chdir(r'N:\ALGORITHMIC MARKETING\FINAL PROJECT')

In [2]:
import credentials  # Import api/access_token keys from credentials.py
#import settings # Import related setting constants from settings.py 

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob
# Streaming With Tweepy 
# http://docs.tweepy.org/en/v3.4.0/streaming_how_to.html#streaming-with-tweepy


# Override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    '''
    Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
    https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
    '''
    
    def on_status(self, status):
        '''
        Extract info from tweets
        '''
        
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)    # Pre-processing the text  
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description,\
            user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False

In [8]:
TRACK_WORDS = ['Red wine']
TABLE_NAME = "Tweets"
TABLE_ATTRIBUTES = "id_str VARCHAR(255), created_at DATETIME, text VARCHAR(255), \
            polarity INT, subjectivity INT, user_created_at VARCHAR(255), user_location VARCHAR(255), \
            user_description VARCHAR(255), user_followers_count INT, longitude DOUBLE, latitude DOUBLE, \
            retweet_count INT, favorite_count INT"

In [9]:
def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [10]:
mydb = mysql.connector.connect(
    host="127.0.0.1",
    user="nikhil",
    passwd="password",
    database="TwitterDB",
    charset = 'utf8',
    auth_plugin='mysql_native_password'
    #port = '3306'
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(TABLE_NAME, TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()

In [11]:
auth  = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [13]:

myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
mydb.close()

When the wedding photographer captures cute photos in the confetti line rather than when we’re smashed on red wine… https://t.co/l4adjn07UL
Long: None, Lati: None
You know what, I think i'm going to have a bottle of red wine tonight. I deserve it
Long: None, Lati: None
Red Red Wine 🎶
Long: None, Lati: None
Enjoy a selection of our favourite dishes from the comfort of your own home, including out pan seared AAA beef tend… https://t.co/zWAQDaK2ms
Long: None, Lati: None
I love #wine reviews by @mgodello
"The glycerin &amp; intensity of salty tang meets a beautiful foxiness for a collabor… https://t.co/CILcTWWn8H
Long: None, Lati: None
@francesbarber13 Why would you add mushroom to a chilli?? Red wine and a bit of tomato purée yes it will be fine!
Long: None, Lati: None
Spaghetti and minced meat, plus red wine. Sweet red.
Long: None, Lati: None
So, the Stellenbosch mafias ain’t opening for wine??? A mere barrel of red wine???
Long: None, Lati: None
Just send out the osteological report (in

@learnmegood I omitted the gin and red wine to keep it family friendly. 😊
Long: None, Lati: None
Well done President Cyril but I have one question: How is cigarette a priority more than RED wine?
Long: None, Lati: None
RT @Harrys_BLN: I miss the simpler times like when harry was drinking straight up tequila out of a wine glass and then Lizzo started chuggi…
Long: None, Lati: None
I have gone almost four days without red wine we’re breaking that today turn it [snail mail’s lush album] up!!!
Long: None, Lati: None


KeyboardInterrupt: 